In [4]:
from os import listdir
from os.path import isfile, join
import pandas as pd

# Datasets filenames
directory = "ados_datasets/"
exclude = [

]
files = [join(directory, f) for f in listdir(directory) \
         if isfile(join(directory, f)) \
         and f.startswith("ados") and f.endswith(".txt") and f not in exclude]
#print(files)

concat_df = pd.DataFrame()  # Concatenation of clean dataframes
for f in files:
    df = pd.read_table(f, header=1, sep="\t")  # Read dataset using 2nd row values as columns headers
    
    # Delete columns with more than 25% of NaN values
    for col in sorted(df.columns):
        count_nan = df[col].isnull().sum()
        ratio_nan = count_nan / len(df)
        if ratio_nan > 0.25:
            del df[col]
    
    # Remove duplicate columns (just keep one of them)
    for col in sorted(df.columns):
        if col.endswith(".1"):
            if col[:-2] in df:
                del df[col]
            else:
                df.rename(columns = {col : col[:-2]}, inplace=True)
        
    # Concatenate dataframes
    if concat_df.empty:
        concat_df = df
    else:
        concat_df = pd.concat([concat_df, df], join="inner", ignore_index=True)
    
    print("Adding file: {}".format(f))
    print("Total Columns: {}".format(len(concat_df.columns)))

# Results
print("\nTOTAL ROWS:", len(concat_df))
print("TOTAL COLUMNS:", len(concat_df.columns))

print("\nCOLUMNS:")
for col in sorted(concat_df.columns):
    print(col)
    
concat_df

Adding file: ados_datasets/ados1_201201_2368.txt
Total Columns: 46
Adding file: ados_datasets/ados2_201201_1250.txt
Total Columns: 15
Adding file: ados_datasets/ados2_200102_9.txt
Total Columns: 10
Adding file: ados_datasets/ados2_200102.txt
Total Columns: 10
Adding file: ados_datasets/ados2_200701_1946.txt
Total Columns: 10
Adding file: ados_datasets/ados1_200701_1250.txt
Total Columns: 10
Adding file: ados_datasets/ados3_200101_1250.txt
Total Columns: 4
Adding file: ados_datasets/ados1_201201_1946.txt
Total Columns: 4
Adding file: ados_datasets/ados2_201201_2368.txt
Total Columns: 4
Adding file: ados_datasets/ados1_200102_9.txt
Total Columns: 4
Adding file: ados_datasets/ados2_201201_19.txt
Total Columns: 4
Adding file: ados_datasets/ados1_200102.txt
Total Columns: 4
Adding file: ados_datasets/ados2_200102_1250.txt
Total Columns: 4
Adding file: ados_datasets/ados1_201201_19.txt
Total Columns: 4
Adding file: ados_datasets/ados3_201201_1946.txt
Total Columns: 4
Adding file: ados_datase

,collection_id,dataset_id,collection_title,promoted_subjectkey
0,2368,11014,Clinical and Immunological Investigations of S...,NDARAB989DG5
1,2368,11014,Clinical and Immunological Investigations of S...,NDARAB989DG5
2,2368,11014,Clinical and Immunological Investigations of S...,NDARAB989DG5
3,2368,11014,Clinical and Immunological Investigations of S...,NDARAB989DG5
4,2368,11014,Clinical and Immunological Investigations of S...,NDARAC787ABV
5,2368,11014,Clinical and Immunological Investigations of S...,NDARAJ572PGT
6,2368,11014,Clinical and Immunological Investigations of S...,NDARAJ572PGT
7,2368,11014,Clinical and Immunological Investigations of S...,NDARAJ572PGT
8,2368,11014,Clinical and Immunological Investigations of S...,NDARAJ572PGT
9,2368,11014,Clinical and Immunological Investigations of S...,NDARAK814FH0
